# CFFI - C Foreign Function Interface


# Goal: call C code from Python without learning a 3rd language

 * SWIG requires to you to learn some additional syntax to create interface files


# Two Modes

 * Application Binary Interface (ABI)
 * Application Progamming Interface (API) -- **Preferred**


# ABI Example

Calls an *existing* C library.  In this case, we'll create it.  First, write a C file:


In [ ]:
%%file mkl_multiply.c
#include "mkl.h"

void mkl_multiply(const int n, const double *x, const double *y, double *result){
    const MKL_INT n_ = (const MKL_INT) n;
    vdMul(n, x, y, result);
}

Compile the C file into a shared library


In [ ]:
%%bash
gcc mkl_multiply.c -shared -o mkl_multiply.so -I${CONDA_PREFIX}/include -L${CONDA_PREFIX}/lib  -lmkl_rt -fPIC

In [ ]:
%ls mkl_multiply*

# ABI Example (cont.)

Import FFI, load library, and define functions to be shared between Python and C


In [ ]:
from cffi import FFI
ffi = FFI()
mkl = ffi.dlopen('./mkl_multiply.so')
ffi.cdef("""
         void mkl_multiply(const int n, const double *x, const double *y, double *result);
""")

Import numpy, create some arrays, and cast them to C data types.


In [ ]:
import numpy as np

N = 100

x = np.arange(N, dtype=np.double)
y = np.arange(N, dtype=np.double)
result = np.empty_like(x)

x_ = ffi.cast("const double*", ffi.from_buffer(x))
y_ = ffi.cast("const double*", ffi.from_buffer(y))
result_ = ffi.cast("double *", ffi.from_buffer(result))

# ABI Example (cont.)

Test the function


In [ ]:
%%timeit
mkl.mkl_multiply(N,  x_, y_, result_)

Compare the numpy


In [ ]:
%timeit x*y

Compare to pure Python


In [ ]:
%%timeit
for i in range(N):
    result[i] = x[i] * y[i]

# API Example

Create a "builder" Python script that includes the C source and the shared Python/C definitions, that is compiled into a single wrapper.


In [ ]:
%%file mkl_multiply_builder.py
import os
from cffi import FFI
ffibuilder = FFI()

ffibuilder.set_source("mkl_multiply",
   r"""
   #include "mkl.h"

   void mkl_multiply(const int n, const double *x, const double *y, double *result){
        const MKL_INT n_ = (const MKL_INT) n;
        vdMul(n_, x, y, result);
   }
   """, 
   library_dirs=[os.environ['CONDA_PREFIX'] + '/lib'],
   include_dirs=[os.environ['CONDA_PREFIX'] + '/include'],
   libraries=['mkl_rt'])

ffibuilder.cdef("""
    void mkl_multiply(const int n, const double *x, const double *y, double *result);
""")

if __name__ == "__main__":
    ffibuilder.compile(verbose=True)

# API Example (cont.)

Run the builder script


In [ ]:
%run mkl_multiply_builder.py

In [ ]:
%ls mkl_multiply*

# API Example (cont.)

Import the the library.  Notice there is no call to `dlopen()`.


In [ ]:
from mkl_multiply import ffi
from mkl_multiply import lib as mkl

Test the function.


In [ ]:
%%timeit
mkl.mkl_multiply(N, x_, y_, result_)

In [ ]:
result

# More Information

 * [CFFI Documentation](https://cffi.readthedocs.io/en/latest/)


In [1]:
%%javascript
function hideElements(elements, start) {
    for(var i = 0, length = elements.length; i < length;i++) {
        if(i >= start) {
            elements[i].style.display = "none";
        }
    }
}

var prompt_elements = document.getElementsByClassName("prompt");
hideElements(prompt_elements, 0)

<IPython.core.display.Javascript object>